In [1]:
import pandas as pd
import re
import numpy as np
import functions
import datetime

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x00000275D6EF06A8; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


### Tabla Detalle

In [13]:
prueba = functions.detalle(cur)

In [15]:
prueba.to_excel('./output/detalle_cartera_20200127.xlsx',encoding='CP1252',index=False)

# Base

In [3]:
df = functions.base(cur)

In [4]:
print('Filas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filas Cartera: 1274
Total ventas: 1274
Total cancelaciones: 44
Total assets: 1142
Total bajas: 218
Total assets activos: 924


In [6]:
df.to_excel('./output/df.xlsx',encoding='CP1252',index=False)

# VENTAS

In [33]:
def ass_vta_fam_mes(df,canal=0):
    idx = pd.period_range('2018-10-01',df.purchase_date__c.max(),freq='M')
    if canal == 1:
        ass_vta_canal_fam_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.to_period("M"),df.canal_venta,df.Family]).sum()
        ass_vta_canal_fam_mes.reset_index(inplace=True)
        ass_vta_canal_fam_mes.set_index('purchase_date__c', inplace=True)
        return ass_vta_canal_fam_mes.pivot_table(index='purchase_date__c', columns=['canal_venta','Family'], values='isSold',fill_value=0).T
    elif canal == 0:
        ass_vta_fam_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.to_period("M"),df.Family]).sum()
        ass_vta_fam_mes.reset_index(inplace=True)
        ass_vta_fam_mes.set_index('purchase_date__c', inplace=True)
        return ass_vta_fam_mes.pivot_table(index='purchase_date__c', columns='Family', values='isSold',fill_value=0).T
    elif canal == 2:
        ass_vta_fam_mes = df[['isSold']].groupby(by=[df.purchase_date__c.dt.to_period("M")]).sum()
        ass_vta_fam_mes.reset_index(inplace=True)
        ass_vta_fam_mes.set_index('purchase_date__c', inplace=True)
        return ass_vta_fam_mes.T     
        


In [34]:
ass_vta_fam_mes(df,canal=2)

purchase_date__c,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
isSold,2,3,24,34,35,56,54,41,62,137,105,23,166,173,110,131,118


In [29]:
functions.ass_vta_fam_mes(df,1)

purchase_year              2018            2019                                \
purchase_month                9 10  11  12    1   2   3   4   5    6    7   8   
canal_venta   Family                                                            
DHO           X FIBER         0  0   3   3    4   8   1   2   9    5   13   3   
              X MUSIC         0  2   1   3    3   7   1   0   0    0    0   0   
              X PRIVACY       0  0   5   0    0  12  14   2  12   13    6   2   
              X PROTECTION    0  0   0   1    2   5  10   3   4   28   20   5   
              X SECURITY      0  0   0   0    0   0   0   0   2    2    2   0   
              X UCOM          0  0   5   0   19  21   1  12  18   56   51   7   
IT integrator X FIBER         0  0   1   4    3   0   1   6   4   11    3   1   
              X MUSIC         0  0   0   0    1   0   0   0   1    0    0   0   
              X PRIVACY       0  0   0   2    0   0   0   0   0    0    0   0   
              X PROTECTION    0  0   0   0    0   0   0   3   0    0    0   0   
              X SECURITY      0  0   0   2    2   0   0   0   3   11    3   0   
              X UCOM          0  0   0   6    0   1   1   3   1    0    0   0   
WEB           X FIBER         0  0   0   1    0   2   0   0   0    1    1   0   
              X MUSIC         2  1   1   0    0   0   1   0   5    0    0   0   
              X PRIVACY       0  0   0   2    0   0   0   2   0    0    0   0   
              X PROTECTION    0  0   0   3    0   0  15   7   0    1    0   0   
              X SECURITY      0  0   0   0    0   0   0   0   0    0    0   0   
              X UCOM          0  0   8   7    1   0   9   1   3    9    6   5   
All                           2  3  24  34   35  56  54  41  62  137  105  23   

purchase_year                                  2020   All  
purchase_month                9   10   11   12    1        
canal_venta   Family                                       
DHO           X FIBER         4    0    1    1    1    58  
              X MUSIC         3    0    0    0    1    21  
              X PRIVACY       9    6    0    2   11    94  
              X PROTECTION   13   27   22    4    9   153  
              X SECURITY     28    7    2    2    0    45  
              X UCOM         46   77   75  104   71   563  
IT integrator X FIBER        14   17    1    1    8    75  
              X MUSIC         1    0    0    0    0     3  
              X PRIVACY       0    0    0    0    0     2  
              X PROTECTION    7    0    4    0    0    14  
              X SECURITY     25   21    0    0    0    67  
              X UCOM          0    0    0    0    0    12  
WEB           X FIBER         2    0    0    1   11    19  
              X MUSIC         1    0    0    0    0    11  
              X PRIVACY       0    0    0    0    0     4  
              X PROTECTION    0    1    2    0    0    29  
              X SECURITY      0    0    0    0    1     1  
              X UCOM         13   17    3   16    5   103  
All                         166  173  110  131  118  1274

In [7]:
functions.ass_vta_prod_mes(df)

purchase_year                   2018            2019                       \
purchase_month                     9 10  11  12    1   2   3   4   5    6   
Family       product_name                                                   
X FIBER      X Fiber 100 - neba    0  0   0   0    1   2   0   1   2    0   
             X Fiber 100 - own     0  0   0   4    3   2   0   4   3   12   
             X Fiber 200 - neba    0  0   1   1    0   3   1   2   2    4   
             X Fiber 200 - own     0  0   0   1    0   0   0   0   6    0   
             X Fiber 50 - neba     0  0   1   1    1   3   0   0   0    0   
             X Fiber 50 - own      0  0   2   1    2   0   1   1   0    1   
X MUSIC      Assets                2  3   2   3    4   7   2   0   6    0   
X PRIVACY    X Privacy 100         0  0   0   0    0   2   0   2   0    3   
             X Privacy 200         0  0   0   1    0   0   0   0   4    5   
             X Privacy 50          0  0   5   3    0  10  14   2   8    5   
X PROTECTION Assets                0  0   0   4    2   5  25  13   4   29   
X SECURITY   X Security 100        0  0   0   2    2   0   0   0   5   11   
             X Security 200        0  0   0   0    0   0   0   0   0    2   
X UCOM       Teamwork+PBX          0  0  13  13   20  22  11  16  22   65   
All                                2  3  24  34   35  56  54  41  62  137   

purchase_year                                                2020   All  
purchase_month                     7   8    9   10   11   12    1        
Family       product_name                                                
X FIBER      X Fiber 100 - neba    1   1    1   13    0    1    1    24  
             X Fiber 100 - own     1   0   15    4    0    0    0    48  
             X Fiber 200 - neba    5   0    2    0    0    0    4    25  
             X Fiber 200 - own     2   1    2    0    1    2    2    17  
             X Fiber 50 - neba     4   1    0    0    1    0    0    12  
             X Fiber 50 - own      4   1    0    0    0    0   13    26  
X MUSIC      Assets                0   0    5    0    0    0    1    35  
X PRIVACY    X Privacy 100         0   0    0    2    0    2    0    11  
             X Privacy 200         1   0    4    4    0    0   10    29  
             X Privacy 50          5   2    5    0    0    0    1    60  
X PROTECTION Assets               20   5   20   28   28    4    9   196  
X SECURITY   X Security 100        2   0   50   27    0    2    0   101  
             X Security 200        3   0    3    1    2    0    1    12  
X UCOM       Teamwork+PBX         57  12   59   94   78  120   76   678  
All                              105  23  166  173  110  131  118  1274

In [8]:
functions.cli_vta_fam_mes(df,0)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5   6   7   8   9  10  11   
canal_venta                                                                   
DHO               0  2   6   7    9  23  14   7  15  34  43  12  35  40  36   
IT integrator     0  0   1   4    3   1   1   8   3   5   4   1   7   2   2   
WEB               1  1   4   8    1   1   7   4   7   6   4   3   9   7   4   
All               1  3  11  19   13  25  22  19  25  45  51  16  51  49  42   

purchase_year      2020  All  
purchase_month  12    1       
canal_venta                   
DHO             34   30  347  
IT integrator    1    1   44  
WEB              3    5   75  
All             38   36  466

# Altas

In [9]:
df_altas = functions.base_altas(df)

In [10]:
df_altas.head(2)

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1


In [13]:
df_altas.to_excel('./detalle_altas.xlsx',encoding='CP1252',index=False)

In [11]:
functions.cli_altas_bajas_mes(df_altas,canal=0)

year     2018        2019                                             2020
month      10 11  12   1   2   3   4   5   6   7   8   9   10  11  12   1 
isClient    3  5  18   13  20  14  18  21  36  36  21  40  44  39  25   31
isBaja      0  0   1    3   0   2   6   8   5   4   5   8  10   7   7    7

In [12]:
altas, bajas = functions.ass_altas_bajas_fam_mes(df, nivel ='Fam',canal = 1)
display(bajas)

year                       2018       2019                                    \
month                        10 11 12    1  2  3   4   5   6   7   8   9  10   
canal_venta   Family                                                           
DHO           X FIBER         0  0  0    0  0  1   0   0   0   3   3   1   1   
              X MUSIC         0  0  0    1  0  1   1   0   0   0   0   0   0   
              X PRIVACY       0  0  0    0  0  0   4   4   2   2  10   2   5   
              X PROTECTION    0  0  0    0  0  0   0   2   3   0   1   1   1   
              X SECURITY      0  0  0    0  0  0   0   0   0   2   0   0   3   
              X UCOM          0  0  0    3  1  6   1   1   4  10   5   5   9   
IT integrator X FIBER         0  0  0    0  0  0   0   0   0   0   0   2   0   
              X MUSIC         0  0  0    0  0  0   0   1   0   0   0   1   0   
              X PRIVACY       0  0  0    0  0  0   0   0   0   0   0   0   0   
              X PROTECTION    0  0  0    0  0  0   1   0   1   0   0   1   0   
              X SECURITY      0  0  0    0  0  0   0   0   0   0   0   0   0   
              X UCOM          0  0  0    0  0  0   0   2   0   0   0   0   0   
WEB           X FIBER         0  0  0    0  0  1   0   0   1   0   0   0   1   
              X MUSIC         0  0  1    0  0  0   1   1   0   0   1   0   0   
              X PRIVACY       0  0  0    0  0  0   2   0   0   0   0   0   0   
              X PROTECTION    0  0  0    2  0  0   1   1   0   0   0   0   0   
              X SECURITY      0  0  0    0  0  0   0   0   0   0   0   0   0   
              X UCOM          0  0  0    0  1  0   0   0   3   1   4  14   5   
All                           0  0  1    6  2  9  11  12  14  18  24  27  25   

year                               2020  All  
month                       11  12    1       
canal_venta   Family                          
DHO           X FIBER        0   1    1   11  
              X MUSIC        0   0    0    3  
              X PRIVACY      0   0    0   29  
              X PROTECTION   2   2    6   18  
              X SECURITY     0   3    0    8  
              X UCOM         4  11   20   80  
IT integrator X FIBER        0   1    0    3  
              X MUSIC        0   0    0    2  
              X PRIVACY      0   0    0    0  
              X PROTECTION   4   0    0    7  
              X SECURITY     0   0    1    1  
              X UCOM         0   0    0    2  
WEB           X FIBER        0   0    1    4  
              X MUSIC        0   0    0    4  
              X PRIVACY      0   0    0    2  
              X PROTECTION   7   0    0   11  
              X SECURITY     0   0    0    0  
              X UCOM         3   2    0   33  
All                         20  20   29  218

## Migras

In [3]:
mig_df=functions.migBase(cur)
mig_df.head(2)

,assetid,assethijo_mig,product_name,Family,cif,canal_venta,mig_date,tipo_migra
0,02i0N00000I3wptQAB,None,Teamwork+PBX,X UCOM,B98115520,DHO,2019-05-09 17:21:03,IN
1,02i0N00000KmJaoQAF,None,Teamwork+PBX,X UCOM,B78909611,DHO,2019-07-19 16:34:33,IN


In [45]:
migin, migout = functions.mig_producto_mes(mig_df)

In [46]:
migin

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      1.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0     11.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      2.0      1.0      1.0   
IT integrator X UCOM  Teamwork+PBX            1.0      0.0      0.0      0.0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0  
              X UCOM  Teamwork+PBX            3.0      1.0      0.0      1.0  
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0

In [47]:
migout

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      1.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0     11.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      2.0      1.0      1.0   
IT integrator X UCOM  Teamwork+PBX            1.0      0.0      0.0      0.0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0  
              X UCOM  Teamwork+PBX            3.0      1.0      0.0      1.0  
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0